# Clone and install specified Python module

WARNING: We assume that `module` name is the same as the `repo` name.

In [1]:
import os
import subprocess
import importlib
from time import sleep

CHECK = "\u2705"
FAIL  = "\u274C"
WARN  = "\u26A0"

if IN_COLAB: 
    repo     = GITHUB_REPO
    user     = GITHUB_USER
    folders  = GITHUB_FOLDERS
    
    homedir  = %pwd
    repodir  = f'{homedir}/{repo}'

    try:
        devnull = '' if GITHUB_VERBOSE else '> /dev/null 2>&1'
    except:
        devnull = '> /dev/null 2>&1'

    print()
    print('BEGIN{clone and install}')
    
    url      = f'https://github.com/{user}/{repo}'
    raw_url  = url.replace('github.com', 'raw.githubusercontent.com')
    raw_url += '/refs/heads/main/'
    toml_file= 'pyproject.toml'

    # 0. uninstall module
    %pip uninstall -y {repo} {devnull}
    
    # 1. delete local repo folder if it exists
    print(f' 1. delete {repo}')
    %rm -rf {repo}

    # 2. clone repo
    print(f' 2. clone {repo}')
    !git clone --depth 1 --filter=blob:none --sparse {url} {devnull}

    # 3. cd to repo
    print(f' 3. cd {repo}')
    sleep(0.5)
    %cd {repodir}
    for folder in folders:
        print(f' 4. check out {folder}')
        !git sparse-checkout set {folder} {devnull}

    # 5. get toml file
    print(f' 5. wget {toml_file}')
    URL = raw_url + toml_file
    !wget -q {URL}  -O {toml_file} {devnull}

    # 6. install module
    print(f' 6. pip install -e .')
    %pip install -e . {devnull}
    try:
        mod = importlib.import_module(repo)
        print(f"{CHECK} Successfully imported {repo}")
    except ModuleNotFoundError as e:
        print(f"{FAIL} Failed to import '{repo}': {e}")

    # 7. return to home folder
    print(f' 7. cd {homedir}')      
    print('END{clone and install}')
    print()
    
    %cd {homedir}  